<a href="https://colab.research.google.com/github/lorenafc/MGI/blob/main/AuthorSmallCitiesEncodingandGeocoding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#import libraries
import pandas as pd
import geopandas as gpd
import requests  # To make API requests
import folium
import numpy as np
from folium import Marker

In [2]:
from geopandas.tools import geocode

In [3]:
# upload the file from your computer
from google.colab import files
uploaded=files.upload()

Saving additional_authors.xlsx to additional_authors.xlsx


In [4]:
# Assuming the uploaded file is named 'additional_authors.xlsx'
file_name = 'additional_authors.xlsx'

# Read the Excel file
authors_small_cities = pd.read_excel(file_name, engine='openpyxl')

authors_small_cities_copy = authors_small_cities

In [5]:
smalldataset = authors_small_cities_copy

#ENCODING FIX (not all of the names were fixed)

In [8]:
# # Function to fix encoding issues #source chatgpt
### THIS FUNCTION FIXED MOST OF THE LOCATIONS, HOWEVER NOT ALL OF THEM.

#HTML ENCODING INCLUDED

import html  # Import the html module for decoding HTML entities

def fix_encoding(text):
    if not isinstance(text, str):
        return text

    # Decode HTML entities first
    text = html.unescape(text)

    # Define possible encodings
    encodings = ['utf-8', 'latin1', 'windows-1252', 'windows-1250', "latin2", "iso-8859-1", "iso-8859-2",'iso-8859-15', 'macroman', 'ascii', 'cp1250', 'cp1251',
        'cp1253', 'cp1254', 'cp1257', 'iso-8859-2', 'iso-8859-3', 'iso-8859-4',
        'iso-8859-5', 'iso-8859-6', 'iso-8859-7',
        'iso-8859-8', 'iso-8859-9', 'koi8-r', 'koi8-u']

    # Try decoding and encoding combinations
    for enc1 in encodings:
        for enc2 in encodings:
            if enc1 != enc2:
                try:
                    # Try decoding and then encoding with different encodings
                    decoded_text = text.encode(enc2, errors='replace').decode(enc1, errors='replace')
                    # Check if decoded_text makes sense
                    if decoded_text.encode(enc1, errors='replace').decode(enc2, errors='replace') == text:
                        return decoded_text
                except (UnicodeEncodeError, UnicodeDecodeError):
                    continue

    # Try double encoding scenarios
    for enc1 in encodings:
        for enc2 in encodings:
            if enc1 != enc2:
                try:
                    first_pass = text.encode(enc1, errors='replace').decode(enc2, errors='replace')
                    decoded_text = first_pass.encode(enc2, errors='replace').decode(enc1, errors='replace')
                    if decoded_text.encode(enc1, errors='replace').decode(enc2, errors='replace') == text:
                        return decoded_text
                except (UnicodeEncodeError, UnicodeDecodeError):
                    continue

    return text



# Apply the encoding fix function to the columns containing city names
smalldataset['borncity'] = smalldataset['born'].apply(fix_encoding)
smalldataset['deathcity'] = smalldataset['death'].apply(fix_encoding)
smalldataset['activecity'] = smalldataset['active'].apply(fix_encoding)


# Display the first few rows to verify the changes
print(smalldataset[["born",'borncity', "death", 'deathcity']].head(15))

#Example usage:
data = {'text': ['CafÃ©', 'NiÃ±o', 'FranÃ§ais', 'Å tÄ›keÅˆ', 'Å tÄ•tÃ­', 'Ã‰tival-Clairefontaine','ArkhangelÊ¹skaiï¸ aï¸¡ oblastÊ¹',
        'Altdorf b. NuÂŒÃ‚Âˆrnberg', 'AltengruÂŒÃ‚Âˆn', 'AltoÂŒÃ‚Âˆtting',"Amdorf &lt;Detern&gt","ApfelstaÂŒÃ‚Âˆdt", "Argonilla JaÂ©n Spain", "Clus &lt;Gandersheim&gt;"]}
df = pd.DataFrame(data)
df['fixed_text'] = df['text'].apply(fix_encoding)
print(df)






                          born                 borncity                death  \
0                                                                        NaN   
1                                                                    Cieszin   
2                        (Chur                    (Chur                  NaN   
3                     A Capela                 A Capela                  NaN   
4                    A GudiÃ±a                 A Gudiña          Puebla City   
5        Å ariÅ¡skÃ© Sokolovce       Šarišské Sokolovce                  NaN   
6                     Å ibenik                  Šibenik              Šibenik   
7                     Å ibenik                  Šibenik              Šibenik   
8                     Å ibenik                  Šibenik  Cividale del Friuli   
9                  Å id Serbia               Šid Serbia                  NaN   
10                   Å ivetice                 Šivetice           Prochowice   
11                Å kofja Loka          

In [10]:
# Apply the encoding fix function to the columns containing city names
# smalldataset['borncity'] = smalldataset['born'].apply(fix_encoding)
# smalldataset['deathcity'] = smalldataset['death'].apply(fix_encoding)
# smalldataset['activecity'] = smalldataset['active'].apply(fix_encoding)


#renaming the columns of the copy
smalldataset_rename = smalldataset.rename(columns = {'cleanedbirth':'bitrhyear',
                       'cleaneddeath':'deathyear', "finalname":"nameandbirthdeathyear"},
            inplace = True)
smalldataset_cleaned = smalldataset.drop(columns=["originalbirth","originaldeath"])

#change the columns order:
smalldataset_cleaned=smalldataset_cleaned[['starturl', 'bitrhyear', 'deathyear', 'nameandbirthdeathyear', 'georeferenceurl', 'born', 'borncity', 'death', 'deathcity','active', 'activecity']]


In [11]:
# export the result as excel file
file_name = 'smalldataset_author_cleaned_html.xlsx'
smalldataset_cleaned.to_excel(file_name)

In [ ]:
print(smalldataset_cleaned.shape)
col_names=list(smalldataset_cleaned.columns)
print(col_names)

(80802, 11)
['starturl', 'bitrhyear', 'deathyear', 'nameandbirthdeathyear', 'georeferenceurl', 'death', 'born', 'active', 'borncity', 'deathcity', 'activecity']


#GEOCODING

In [ ]:
#souce: chat gpt and PyGEP course

#this code keeps running form many hours and shows no output.


from geopy.geocoders import Nominatim
from geopy.exc import GeopyError
import time
import logging
from multiprocessing import Pool

# Initialize logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Initialize Nominatim geocoder with a user agent
geolocator = Nominatim(user_agent="my_Geocoder_application")

# Function to geocode city names with error handling and rate limiting
def geocode_city(city_name):
    try:
        # Rate limiting
        time.sleep(1)
        # Attempt to geocode the city name with a timeout
        location = geolocator.geocode(city_name, timeout=10)
        if location:
            return (city_name, location.latitude, location.longitude)
        else:
            logging.warning(f"Geocoding failed for: {city_name}")
            return (city_name, None, None)
    except GeopyError as e:
        logging.error(f"Geocoding error for {city_name}: {e}")
        return (city_name, None, None)

# Function to apply geocoding to a list of city names
def geocode_list(city_list):
    return [geocode_city(city) for city in city_list]

# Function to geocode a large DataFrame with multiprocessing
def geocode_dataframe(df, chunk_size=1000, n_jobs=4):
    # Split the DataFrame into chunks
    chunks = [df[i:i + chunk_size] for i in range(0, len(df), chunk_size)]

    # Prepare the data for parallel processing
    city_lists = [chunk['borncity'].tolist() for chunk in chunks]

    # Use multiprocessing Pool to process chunks in parallel
    results = []
    with Pool(n_jobs) as pool:
        for result in pool.imap_unordered(geocode_list, city_lists):
            results.extend(result)
            logging.info(f"Processed {len(results)} cities so far")

    # Convert results back into a DataFrame
    geocoded_df = pd.DataFrame(results, columns=['borncity', 'latitude', 'longitude'])

    return geocoded_df

# # Example DataFrame for testing
# data = {'bornlocation': ['New York', 'Los Angeles', 'Nonexistent City', "A Capela", "A Gudiña", "Šarišské Sokolovce", "Šibenik", "Škvorci u Českého Brodu","Rasch <Altdorf bei Nürnberg>", "Øvre Eiker", "Rasch (Altdorf bei Nürnberg)"]}
# authors_small_cities_cleaned = pd.DataFrame(data)

# Apply the geocoding function to the DataFrame
authors_small_cities_geocoded = geocode_dataframe(smalldataset_cleaned, chunk_size=100, n_jobs=4)



ERROR:root:Geocoding error for  : Non-successful status code 400
ERROR:root:Geocoding error for  : Non-successful status code 400


In [ ]:
# Display the first few rows to verify the results
print(authors_small_cities_geocoded.head(20))


In [ ]:
# export the result as excel file
file_name = 'author_born_cleaned_html_geocoded.xlsx'
authors_small_cities_geocoded.to_excel(file_name)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd


# Specify the path in Google Drive
file_path = '/content/drive/My Drive/geocoded_authors_small_cities.xlsx'

# Save the DataFrame to an Excel file in Google Drive
authors_small_cities_geocoded.to_excel(file_path, index=False)


In [ ]:
# #if the code above works, add deathcity and activecity

# import pandas as pd
# from geopy.geocoders import Nominatim
# from geopy.exc import GeopyError
# import time
# import logging
# from multiprocessing import Pool

# # Initialize logging
# logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# # Initialize Nominatim geocoder with a user agent
# geolocator = Nominatim(user_agent="my_Geocoder_application")

# # Function to geocode city names with error handling and rate limiting
# def geocode_city(city_name):
#     try:
#         # Rate limiting
#         time.sleep(1)
#         # Attempt to geocode the city name with a timeout
#         location = geolocator.geocode(city_name, timeout=10)
#         if location:
#             return (city_name, location.latitude, location.longitude)
#         else:
#             logging.warning(f"Geocoding failed for: {city_name}")
#             return (city_name, None, None)
#     except GeopyError as e:
#         logging.error(f"Geocoding error for {city_name}: {e}")
#         return (city_name, None, None)

# # Function to apply geocoding to a list of city names
# def geocode_list(city_list):
#     return [geocode_city(city) for city in city_list]

# # Function to geocode a large DataFrame with multiprocessing
# def geocode_dataframe(df, column, chunk_size=1000, n_jobs=4):
#     # Split the DataFrame into chunks
#     chunks = [df[i:i + chunk_size] for i in range(0, len(df), chunk_size)]

#     # Prepare the data for parallel processing
#     city_lists = [chunk[column].tolist() for chunk in chunks]

#     # Use multiprocessing Pool to process chunks in parallel
#     results = []
#     with Pool(n_jobs) as pool:
#         for result in pool.imap_unordered(geocode_list, city_lists):
#             results.extend(result)
#             logging.info(f"Processed {len(results)} cities for column '{column}' so far")

#     # Convert results back into a DataFrame
#     geocoded_df = pd.DataFrame(results, columns=[column, f'{column}_latitude', f'{column}_longitude'])

#     return geocoded_df

# # Example DataFrame for testing
# data = {
#     'bornlocation': ['New York', 'Los Angeles', 'Nonexistent City', "A Capela", "A Gudiña", "Šarišské Sokolovce", "Šibenik", "Škvorci u Českého Brodu","Rasch <Altdorf bei Nürnberg>", "Øvre Eiker", "Rasch (Altdorf bei Nürnberg)"],
#     'deathlocation': ['New York', 'Los Angeles', 'Nonexistent City', "A Capela", "A Gudiña", "Šarišské Sokolovce", "Šibenik", "Škvorci u Českého Brodu","Rasch <Altdorf bei Nürnberg>", "Øvre Eiker", "Rasch (Altdorf bei Nürnberg)"],
#     'activelocation': ['New York', 'Los Angeles', 'Nonexistent City', "A Capela", "A Gudiña", "Šarišské Sokolovce", "Šibenik", "Škvorci u Českého Brodu","Rasch <Altdorf bei Nürnberg>", "Øvre Eiker", "Rasch (Altdorf bei Nürnberg)"]
# }
# authors_small_cities_cleaned = pd.DataFrame(data)

# # Apply the geocoding function to each relevant column
# columns_to_geocode = ['bornlocation', 'deathlocation', 'activelocation']
# geocoded_dfs = []

# for column in columns_to_geocode:
#     geocoded_df = geocode_dataframe(authors_small_cities_cleaned, column, chunk_size=100, n_jobs=4)
#     geocoded_dfs.append(geocoded_df)

# # Merge the geocoded data back into the original DataFrame
# for geocoded_df, column in zip(geocoded_dfs, columns_to_geocode):
#     authors_small_cities_cleaned = authors_small_cities_cleaned.merge(geocoded_df, on=column, how='left')

# # Display the first few rows to verify the results
# print(authors_small_cities_cleaned.head())


In [ ]:
# #trying with chunks for a bigger dataset

# import pandas as pd
# from geopy.geocoders import Nominatim
# from geopy.exc import GeopyError
# import time
# from multiprocessing import Pool

# # Initialize Nominatim geocoder with a user agent
# geolocator = Nominatim(user_agent="my_Geocoder_application")

# # Function to geocode city names with error handling
# def geocode_city(city_name):
#     try:
#         # Attempt to geocode the city name with a timeout
#         location = geolocator.geocode(city_name, timeout=10)
#         if location:
#             return (city_name, location.latitude, location.longitude)
#         else:
#             print(f"Geocoding failed for: {city_name}")
#             return (city_name, None, None)
#     except GeopyError as e:
#         print(f"Geocoding error for {city_name}: {e}")
#         return (city_name, None, None)

# # Function to apply geocoding to a list of city names
# def geocode_list(city_list):
#     return [geocode_city(city) for city in city_list]

# # Function to geocode a large DataFrame with multiprocessing
# def geocode_dataframe(df, chunk_size=1000, n_jobs=4):
#     # Split the DataFrame into chunks
#     chunks = [df[i:i + chunk_size] for i in range(0, len(df), chunk_size)]

#     # Prepare the data for parallel processing
#     city_lists = [chunk['bornlocation'].tolist() for chunk in chunks]

#     # Use multiprocessing Pool to process chunks in parallel
#     with Pool(n_jobs) as pool:
#         results = pool.map(geocode_list, city_lists)

#     # Flatten the list of results
#     flat_results = [item for sublist in results for item in sublist]

#     # Convert results back into a DataFrame
#     geocoded_df = pd.DataFrame(flat_results, columns=['bornlocation', 'latitude', 'longitude'])

#     return geocoded_df




# # Apply the geocoding function to the DataFrame
# # For large datasets, adjust chunk_size and n_jobs according to your system capabilities
# authors_small_cities_geocoded = geocode_dataframe(smalldataset_cleaned, chunk_size=100, n_jobs=4)

# # Display the first few rows to verify the results
# print(authors_small_cities_geocoded.head())

# # If you need to apply geocoding to multiple columns like 'borncity', 'deathcity', 'activecity',
# # you can extend the geocode_chunk function accordingly and modify the DataFrame appropriately.



Geocoding error for  : Non-successful status code 400
Geocoding error for  : Non-successful status code 400
Geocoding failed for: Jerslev (bei Hjørring)
Geocoding failed for: Crosna Schlesien
Geocoding failed for: Crossen (Vogtland)
Geocoding failed for: OberboÂbingen
Geocoding failed for: Crotendorf
Geocoding failed for: Oberbobritzsch bei Freiberg/Sachsen
Geocoding failed for: OberbuÂren
Geocoding failed for: Škvorci u Českého Brodu
Geocoding failed for: Croughton Northampton England
Geocoding failed for: JevÃÂ­ÃâÃÂku
Geocoding failed for: Crowell bei Oxfordshire
Geocoding failed for: Crumpsall Lancashire England
Geocoding failed for: Ober-Ellgut (Schlesien)
Geocoding failed for: Ctesiphon
Geocoding failed for: Oberfarnstedt
Geocoding failed for: Ärakleion
Geocoding failed for: Ägypten (Altertum)
Geocoding failed for: Oberherzogswaldau
Geocoding failed for: Jhamatpur
Geocoding failed for: JiÃÂÃÂ­nÃâºvsi (ÃÅesko)
Geocoding failed for: OberkaÂrnten
Geocoding failed 

Geocoding failed for: Triebel/Lausitz
Geocoding failed for: Trigance (Diocèse de Riez)
Geocoding failed for: Trostberg/Südtirol
Geocoding failed for: Troston Hall in Suffolk
Geocoding failed for: TruÂnzig
Geocoding failed for: Truntlack
Geocoding failed for: Tschigirin
Geocoding failed for: Tschigirin
Geocoding failed for: Tschilesen
Geocoding failed for: Tsiï¸ aï¸¡pina Chashnitski region
Geocoding failed for: TuÂrkei
Geocoding failed for: TuÂrkheim
Geocoding failed for: TuÂrkheim b. Mindelheim
Geocoding failed for: Turebygård
Geocoding failed for: Turia de Jos
Geocoding failed for: Turriès (Alpes-de-Haute-Provence)
Geocoding failed for: Turries b. Sisteron i. d. Provence
Geocoding failed for: Tuy-Lộc Quăng Binh
Geocoding failed for: TwardogoÃÅÃâÃÂra
Geocoding failed for: Tyana Cappadocia Anatolia 
Geocoding failed for: Tyana Cappadocia Anatolia 
Geocoding failed for: Tykeham
Geocoding failed for: Tyrolensis Canaciensis
Geocoding failed for: Tyuron
Geocoding failed for: 

Geocoding failed for: Wadley (Berkshire)
Geocoding failed for: Wahanów
Geocoding failed for: WahrenbruÂck
Geocoding failed for: WahrenbruÂck
Geocoding failed for: WaldmuÂnchen
Geocoding failed for: Waldrems/Allgäu
Geocoding failed for: Waldridge Buckinghamshire
Geocoding failed for: Walgrave Northants
Geocoding failed for: WallduÂrn
Geocoding failed for: Waltershausen Th¼ringen
Geocoding failed for: Waltershausen/Grabfeld - Unterfranken
Geocoding failed for: Walthamstow Essex England
Geocoding failed for: Warbester Hoy
Geocoding failed for: Wardes-sur-Epte in Beauvaisis
Geocoding failed for: Warlin (Strelitz)
Geocoding failed for: Warner Hall Gloucester County Colony of Virginia
Geocoding failed for: Warren Litchfield County Conn.
Geocoding failed for: WassertruÂdingen (Kr.Ansbach)
Geocoding failed for: Wattenwill Switz.
Geocoding failed for: Waverly Northampton England
Geocoding failed for: Weathersfield Conn.
Geocoding failed for: Wechtersbach  Hesse
Geocoding failed for: W

In [ ]:
#### USE small_cities to test the geocoding function



# # Function to geocode the city names source: PyGEO course and chatgpt

# from geopy.geocoders import Nominatim
# from geopy.exc import GeopyError


# # Initialize Nominatim geocoder
# geolocator = Nominatim(user_agent="my_Geocoder_application")

# # Function to geocode city names
# def geocode_city(city_name):
#     try:
#         location = geolocator.geocode(city_name, timeout=10)
#         if location:
#             return (location.latitude, location.longitude)
#         else:
#             print(f"Geocoding failed for: {city_name}")
#             return None
#     except GeopyError as e:
#         print(f"Geocoding error for {city_name}: {e}")
#         return None

# # #Example DataFrame
# # data = {'bornlocation': ['New York', 'Los Angeles', 'Nonexistent City', "A Capela", "A Gudiña", "Šarišské Sokolovce", "Šibenik", "Škvorci u Českého Brodu", "Øvre Eiker"]}
# # authors_small_cities_cleaned = pd.DataFrame(data)

# # # Apply the geocoding function to the 'bornlocation' column
# # authors_small_cities_cleaned['bornlocation_geo'] = authors_small_cities_cleaned['bornlocation'].apply(geocode_city)

# # # Display the first few rows to verify the results
# # print(authors_small_cities_cleaned.head(10))

# # Apply the geocoding function to the 'borncity', 'deathcity', and 'activecity' columns
# small_cities['bornlocation_geo'] = small_cities['bornlocation'].apply(geocode_city)

NameError: name 'small_cities' is not defined